In [1]:
#Program Manual Compilation - written by Benjamin Pardue (benjamin.pardue@guycarp.com) on 5/2/2019
#GUI written 5/8/2019


import glob, os, msvcrt, sys, shutil
import comtypes.client
import win32com.client
from PyPDF2 import PdfFileMerger, PdfFileReader, PdfFileWriter
from tkinter import *
from tkinter import scrolledtext, messagebox, Image, filedialog
from tkinter.ttk import *

def PPTtoPDF(inputFileName, outputFileName, formatType = 32):
    powerpoint = comtypes.client.CreateObject("Powerpoint.Application")
    powerpoint.Visible = 1
    if outputFileName[-3:] != 'pdf':
        outputFileName = outputFileName + ".pdf"
    deck = powerpoint.Presentations.Open(inputFileName)
    deck.SaveAs(outputFileName, formatType) # formatType = 32 for ppt to pdf
    deck.Close()
    powerpoint.Quit()
    
def WordtoPDF(inputFileName, outputFileName, formatType = 17):
    wdFormatPDF = formatType
    word = comtypes.client.CreateObject('Word.Application')
    doc = word.Documents.Open(inputFileName)
    doc.SaveAs(outputFileName, FileFormat=wdFormatPDF)
    doc.Close()
    word.Quit()
    
def XLtoPDF(inputFileName, outputFileName, outputArea = 'A1:G50', sheets = [1]):
    o = win32com.client.Dispatch("Excel.Application")
    o.Visible = False
    wb_path = inputFileName
    wb = o.Workbooks.Open(wb_path)
    ws_index_list = sheets #say you want to print these sheets
    path_to_pdf = outputFileName
    print_area = outputArea
    for index in ws_index_list:
        #off-by-one so the user can start numbering the worksheets at 1
        ws = wb.Worksheets[index - 1]
        ws.PageSetup.Zoom = False
        ws.PageSetup.FitToPagesTall = 1
        ws.PageSetup.FitToPagesWide = 1
        ws.PageSetup.PrintArea = print_area
    wb.WorkSheets(ws_index_list).Select()
    wb.ActiveSheet.ExportAsFixedFormat(0, path_to_pdf)

def MovePDF(inputFileName, outputFileName):
    #os.rename(inputFileName, outputFileName)
    shutil.copyfile(inputFileName, outputFileName)
    
def FolderToPDF(inputFolder, outputFolder):
    lst = os.listdir(inputFolder)
    lst.sort()
    for files in lst:
        try:
            if files.endswith(".pdf") and not files.startswith('.'):
                shutil.copyfile(inputFolder + '\\' + files,outputFolder + '\\' + files)
                print('converted '+str(files))
            elif files.endswith(".docx") and not files.startswith('.'):
                WordtoPDF(inputFolder + '\\' + files,outputFolder + '\\' + files[0:-4]+'pdf')
                print('converted '+str(files))
            elif files.endswith('.pptx') and not files.startswith('.'):
                PPTtoPDF(inputFolder + '\\' + files,outputFolder + '\\' + files[0:-4]+'pdf')
                print('converted '+str(files))
            elif files.endswith('.xlsx') and not files.startswith('.'):
                XLtoPDF(inputFolder + '\\' + files,outputFolder + '\\' + files[0:-4]+'pdf',excelCellRange,excelSheetRange)
            else:
                print('could not move ' + str(files))
        except: messagebox.showinfo("Status",files+" caused a conversion error")
            
def append_PDFs(inputFolder1, outputFile):
    lst = os.listdir(inputFolder1)
    lst.sort()
    x = [a for a in lst if a.endswith(".pdf")]
    merger = PdfFileMerger()
    for pdf in x:
        try:
            filePath = inputFolder1+'\\'+pdf
            merger.append(open(filePath, 'rb'))
            print('appended '+str(pdf))
        except: messagebox.showinfo("Status",pdf+" caused an appending error")
    with open(outputFile, "wb") as fout:
        merger.write(fout)
        
def add_bookmarks(inputFolder2, outputFile):
    bookmarkee = PdfFileMerger()
    bookmarkee.append(open(outputFile, 'rb'))
    lst = os.listdir(inputFolder2)
    lst.sort()
    j = 0
    for i in lst:
        try:
            pdfFile = PdfFileReader(open(inputFolder2+'\\'+i, 'rb'))
            pageNum = pdfFile.getNumPages()
            bookmarkee.addBookmark(i, j)
            print(i+" bookmarked")
            j = j + pageNum
        except: messagebox.showinfo("Status",i+"Problem with the folder")
    bookmarkee.write(outputFile[0:-11]+'_RenameBookmarksAndSave.pdf')
    
def construct_program_manual(folderOG):
    newPath = folderOG+'\\as_PDFs'
    if not os.path.exists(newPath):
        os.makedirs(newPath)
        messagebox.showinfo("Status","created " + newPath+ " to save PDF versions")
    else: messagebox.showinfo("Status",newPath+ " identified")
    FolderToPDF(folderOG,newPath)
    append_PDFs(newPath,folderOG+'\\TempPM.pdf')
    add_bookmarks(newPath, folderOG+'\\TempPM.pdf')

window = Tk()
window.title('Program Manual Compilation')
l1 = Label(window, text="This compiles every file in a folder into one PDF,",font=("Arial Bold",12))
l1.pack()
l2 = Label(window, text="complete with Bookmarks.",font=("Arial Bold",12))
l2.pack()
l3 = Label(window, text="This version only supports PDF, Excel, Word, and PPT Files",font=("Arial Bold",12))
l3.pack()
l4 = Label(window, text="Other filetypes should be converted",font=("Arial Bold",12))
l4.pack()
l5 = Label(window, text="Once the program is complete, double-check",font=("Arial Bold",12))
l5.pack()
l6 = Label(window, text="the bookmark labels in the output PDF",font=("Arial Bold",12))
l6.pack()
l7 = Label(window, text="Please input the range of excel cells and sheets (if applicable)",font=("Arial Bold",12))
l7.pack()

cellRange=Entry(window,width=12)
cellRange.insert(END, 'A1:G50')
cellRange.pack()
#sheetRange=Entry(window,width=12)
#sheetRange.insert(END, '[1,2,3]')
#sheetRange.pack()

#Step2(IF YOU HAVE EXCEL FILES): define the range of cells to be dumped into the PDF and which sheets
appendBtn = Button(window, text='Select Folder', command=lambda:clicked())
quitBtn = Button(window, text='Exit Program', command=window.quit)
appendBtn.pack()
quitBtn.pack()


def clicked():
    try:
        window.filename =  filedialog.askdirectory()
        filePath1 = window.filename
        #filePath1 = filePath1.Replace("\\","/")
    except: messagebox.showinfo("Status","Problem with the folder")
    try:
        cellRange1 = str(cellRange.get())
    except: messagebox.showinfo("Status","You entered no cell range, so default values used")
    construct_program_manual(filePath1)
    messagebox.showinfo("Status","You're bookmarked file is at "+filePath1+'\_RenameBookmarksAndSave.pdf')
    window.quit()
    sys.exit()

    
window.mainloop()
print('Compilation into one PDF complete')

converted AdvantagePoint CenterPoint Accumulation Analysis Quick Start Guide.pdf
converted AdvantagePoint Deterministic Scenarios Quick Start Guide.pdf
converted AdvantagePoint Events Quick Start Guide.pdf
converted AdvantagePoint Hazards QuickStartGuide.pdf
converted AdvantagePoint Overview Quick Start Guide.pdf
converted AdvantagePoint Porfolio Selection Quick Start Guide.pdf
converted AdvantagePoint Underwriting Quick Start Guide wAAL.pdf
converted AdvantagePoint Underwriting Quick Start Guide.pdf
converted Tropical Storm Quick Start Guide.pdf
converted WC Overview Quick Start Guide.pdf
converted WC Underwriting Quick Start Guide.pdf
converted Wildfire Quick Start Guide.pdf
could not move as_PDFs
appended AdvantagePoint CenterPoint Accumulation Analysis Quick Start Guide.pdf
appended AdvantagePoint Deterministic Scenarios Quick Start Guide.pdf
appended AdvantagePoint Events Quick Start Guide.pdf
appended AdvantagePoint Hazards QuickStartGuide.pdf
appended AdvantagePoint Overview Qui

SystemExit: 